
## 네이버 기사 데이터 수집
    - scrapy에서 selenium 사용하기

In [1]:
from PIL import Image as pil
from selenium import webdriver
import pandas as pd
import numpy as np
import scrapy
import requests
import json
from scrapy.http import TextResponse
from selenium import webdriver
from bs4 import BeautifulSoup

### 1.프로젝트 생성

In [2]:
!scrapy startproject naver_article

Error: scrapy.cfg already exists in C:\code\Python_Folder\02_MYSQL\06_scrapy\off\naver_article


In [2]:
!tree naver_article

폴더 PATH의 목록입니다.
볼륨 일련 번호는 5C51-42FB입니다.
C:\CODE\PYTHON_FOLDER\02_MYSQL\06_SCRAPY\OFF\NAVER_ARTICLE
└─naver_article
    ├─spiders
    │  └─__pycache__
    └─__pycache__


### 2. xpath찾기
- article links : [not()] 사용
- article content

In [3]:
from fake_useragent import UserAgent

In [4]:
ua=UserAgent().chrome

In [5]:
url='https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=101'
headers = {'user-agent' : ua}
req = requests.get(url,headers=headers)
req

<Response [200]>

In [6]:
response= TextResponse(req.url, body=req.text, encoding='utf-8')

In [7]:
links = response.xpath('//*[@id="main_content"]/div[2]/ul/li/dl/dt[not(@class="photo")]/a/@href').extract() # 썸네일이 없는 기사는 두번째 dt가 없으므로 19개가 나온다
len(links), links[0]  # [not(@class="photo")] 클래스 photo가 없는 것을 선택

(20,
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=015&aid=0004450522')

##### selenium 사용 : xpath

In [8]:
options = webdriver.ChromeOptions()
options.add_argument("headless")
driver = webdriver.Chrome(options=options)

In [9]:
driver.get(url)

In [10]:
elements = driver.find_elements_by_xpath('//*[@id="main_content"]/div[2]/ul/li/dl/dt[not(@class="photo")]/a')
len(elements)

#links = response.xpath('//*[@id="main_content"]/div[2]/ul/li/dl/dt[not(@class="photo")]/a/@href').extract()

20

In [11]:
# get_attribute("href") element안에 있는 'href'주소를 속성값으로 가져와 WebDriver로 접속한다.

In [12]:
links = [element.get_attribute("href") for element in elements] 
len(links), links[0]                                              

(20,
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=015&aid=0004450522')

In [13]:
link = links[0]
link

'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=015&aid=0004450522'

In [14]:
driver.quit()

##### content 

In [15]:
req = requests.get(link, headers=headers)
response = TextResponse(req.url, body=req.text, encoding="utf-8")

In [16]:
title = response.xpath('//*[@id="articleTitle"]/text()')[0].extract()
category = response.url.split("sid1=")[1].split("&")[0] #split("sid1") sid1을 기준으로 리스트 구분
content = response.xpath('//*[@id="articleBodyContents"]/text()').extract()
content = " ".join(content).strip() #strip() 문자 양끝을 정리
title, category, content

('저금리에 코로나까지…이중고 시달리는 보험업계 "수익성 제고해야"',
 '101',
 '저금리에 신종 코로나바이러스 감염증(코로나19)까지 이중고를 겪고 있는 국내 보험산업이 시장 요구이익을 만족시키기 위해서는 현재보다 1.9배의 이익이 필요한 것으로 나타났다. 지난해 말 당기순이익과 비교해 생명보험사는 2.1배, 손해보험사는 1.5배의 이익이 더 필요하다는 분석이다. 보험연구원은 16일 오후 \'제로금리 시대, 보험산업의 영향과 과제\'를 주제로 온라인 세미나를 개최했다. 이번 세미나는 제로금리시대의 보험산업 수익성을 분석하고 수익성 개선을 위한 다양한 대응방안을 논의하기 위해 마련됐다. \'보험산업의 수익성과 대응방안\'을 주제로 발표에 나선 노건엽 보험연구원 연구위원은 "보험회사 당기순이익은 최근 10년간(2010~2019년) 가장 낮은 수준을 나타내며 2017년 이후 하락 추세"라며 "수익성 지표인 자기자본이익률(ROE)은 10년 전에 비해 3분의 1수준으로 하락했다"고 진단했다. 그는 "2019년 기준 자본비용과 ROE 차이를 금액으로 환산하면 추가적인 시장 요구이익은 생보 3조6000억원, 손보 1조1000억원으로 총 4조7000억원"이라고 덧붙였다. 보험산업의 이익이 건강한지를 분석하기 위해 노 연구위원은 내재가치기법(EV)을 활용해 분석했다. 내재가치는 보유계약과 자산 평가익 등을 통해 보험회사 미래이익과 금리에 대한 가치 변화를 파악할 수 있다.  노 연구위원은 "실제 채권 처분이익이 당기순이익에서 차지하는 비중을 보면 2019년 기준 생보 62%, 손보 87%로 보험영업 손실을 투자영업 이익으로 상쇄하고 있다"고 말했다. 이어 그는 "일정 부분 채권 매각도 필요하겠지만 과도한 매각은 미래의 이익을 앞당겨 실현한 것으로 보험산업의 현재 이익구조가 건강하지 않음을 보여주는 단면이라고 할 수 있다"고 설명했다. 또 일부사는 보유계약가치에서 마이너스가 발생할 정도로 금리하락에 따른 영향이 심각하므로 보유계약에 대한 관리방안이 필요하다고 지적했다. 

###  3. items.py

##### load itemts.py  

In [17]:
# %load naver_article/naver_article/items.py
# Define here the models for your scraped items
#
# See documentation in:
# https://docs.scrapy.org/en/latest/topics/items.html

import scrapy


class NaverArticleItem(scrapy.Item):
    # define the fields for your item here like:
    # name = scrapy.Field()
    pass


In [18]:
%%writefile naver_article/naver_article/items.py
import scrapy


class NaverArticleItem(scrapy.Item):
    title = scrapy.Field()
    content = scrapy.Field()
    category = scrapy.Field()
    link = scrapy.Field()
    

Overwriting naver_article/naver_article/items.py


### 4. spider.py 
- selenium 사용

In [19]:
!tree naver_article/ 

#naver_article 디렉토리에 있는 scrapy.cfg파일을 실행

폴더 PATH의 목록입니다.
볼륨 일련 번호는 5C51-42FB입니다.
C:\CODE\PYTHON_FOLDER\02_MYSQL\06_SCRAPY\OFF\NAVER_ARTICLE\
잘못된 경로 - \CODE\PYTHON_FOLDER\02_MYSQL\06_SCRAPY\OFF\NAVER_ARTICLE\

에 하위 폴더가 없습니다.


In [20]:
# Spider실행 -> start_requests 실행 -> start_requests매서드에서 selenium 실행 
# links 수집이되면 parse_content로 넘어간다.

In [22]:
%%writefile naver_article/naver_article/spiders/spider.py
import scrapy
from selenium import webdriver
from naver_article.items import NaverArticleItem 

# naver_article디렉토리에 있는 scrapy.cfg파일 위치를 기준으로 items를 실행하므로 from naver_article.items
# items.py를 전역영역설정을 하면 어디서든 import 가능

class ArticleSpider(scrapy.Spider):
    name = 'NaverArticle'
    
    #allow_domain = ['https://news.naver.com']
    #start_urls = ['https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=100']
    
    #def parse(self, response):
        # selenium
    
    def start_requests(self):
        url = 'https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=101'
        #selenium
        options = webdriver.ChromeOptions()
        options.add_argument("headless")
        driver = webdriver.Chrome(options=options)
        driver.get(url)
        elements = driver.find_elements_by_xpath('//*[@id="main_content"]/div[2]/ul/li/dl/dt[not(@class="photo")]/a')
        links = [element.get_attribute("href") for element in elements] 
        
        for link in links:
            yield scrapy.Request(link, callback=self.parse_content)
            
    def parse_content(self, response):
        item = NaverArticleItem()
        item["title"] = response.xpath('//*[@id="articleTitle"]/text()')[0].extract()
        item["category"] = response.url.split("sid1=")[1].split("&")[0]
        content = response.xpath('//*[@id="articleBodyContents"]/text()').extract() 
        item["content"] = " ".join(content).strip()
        item["link"] = response.url
        yield item
        

Overwriting naver_article/naver_article/spiders/spider.py


In [73]:
%%writefile naver_article/naver_article/spiders/spider.py
import scrapy
from selenium import webdriver
from naver_article.items import NaverArticleItem

class ArticleSpider(scrapy.Spider):
    name = "NaverArticle"
    allow_domain = ["https://news.naver.com"]
    start_urls = ["https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=100"]
    
    def parse(self, response):
        options = webdriver.ChromeOptions()
        options.add_argument("headless")
        driver = webdriver.Chrome(options=options)
        driver.get(response.url)
        elements = driver.find_elements_by_xpath('//*[@id="main_content"]/div[2]/ul/li/dl/dt/a')
        links = [element.get_attribute("href") for element in elements]
        driver.quit()
        for link in links:
            yield scrapy.Request(link, callback=self.parse_content)
            
    def parse_content(self, response):
        item = NaverArticleItem()
        item["title"] = response.xpath('//*[@id="articleTitle"]/text()')[0].extract()
        item["category"] = response.xpath('//*[@id="lnb"]/ul/li[2]/a/@href')[0].extract().split("sid1=")[1]
        content = response.xpath('//*[@id="articleBodyContents"]/text()').extract()
        item["content"] = "".join(content).strip()
        item["link"] = response.url
        yield item

Overwriting naver_article/naver_article/spiders/spider.py


### 5. scrapy 프로젝트 실행
    - windows 사용하는 경우에는 직접 cli 환경에서 실행
    - scrapy crawl NaverArticle -o article.csv
    - robots.txt 문제 > settings.py 수정

In [22]:
!pwd

/c/code/Python_Folder/02_MYSQL/06_scrapy/off


In [23]:
%%writefile run.sh
cd naver_article
scrapy crawl NaverArticle -o article.csv


Overwriting run.sh


In [67]:
!/bin/bash run.sh

지정된 경로를 찾을 수 없습니다.


In [72]:
!scrapy crawl NaverArticle -o article.csv

Scrapy 2.4.0 - no active project

Unknown command: crawl

Use "scrapy" to see available commands


In [75]:
import pandas as pd

In [23]:
df = pd.read_csv(".\\naver_article\\article.csv")
df.tail(2)

,category,content,link,title
18,101,"(서울=연합뉴스) 임수정 기자 = 16일 원/달러 환율이 하락해 1,100원대에 진...",https://news.naver.com/main/read.nhn?mode=LSD&...,"원/달러 환율, 1,100원대 진입…23개월여만에 최저치"
19,101,[아이뉴스24 한수연 기자] 주식회사 씨앤투스성진이 전문가용 컬러마스크인 '아에르 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,"씨앤투스성진, 전문가용 컬러마스크 '아에르 프로' 출시"


### 6. pilelines 사용
- 크롤링한 데이터 mongdb에 저장하기 : pymongo 사용
- 크롤링한 데이터에서 특정한 키워드가 있는 기사가 수집되면 slack메신저로 기사 내용과 링크 전송하기
    - slack 메신져의 incoming webbhook은 1초에 1번 사용가능 > time.sleep(1) 사용